In [1]:
%reset -f

In [2]:
import pacKage
import loaddata
import uNet
from pacKage import *
from uNet import *
from loaddata import *
rootPath = os.getcwd()
dataPath = os.path.join(rootPath,"data")
modelPath = os.path.join(rootPath,"model")
if not os.path.exists(dataPath):
    os.makedirs(dataPath)
if not os.path.exists(modelPath):
    os.makedirs(modelPath)
os.listdir(dataPath)

device is cuda


['desktop.ini', 'label', 'test', 'train', 'WQ643945_satellite.tif']

In [3]:
modules_to_reload = ["pacKage", "uNet", "loaddata"]

    
for module_name in modules_to_reload:
    # Dynamically import the module by name
    module = __import__(module_name)
    # Now reload the module
    importlib.reload(module)

device is cuda


In [4]:
loaddata.__all__

['minmax', 'TrainDataset', 'TestDataset', 'train_batch', 'accuracy']

In [5]:
train_dir = os.path.join(dataPath, "train")
label_dir = os.path.join(dataPath, "label")
test_dir  = os.path.join(dataPath, "test")

**transfer our image data into two types of tensor and the label into another tensor**
1. (350, 350, 4) for infrared, near infrared, cloud mask, elevation. we call that "ince"
2. (350, 350, 3) for RGB, we call that "rgb"
3. (350, 350, 1) for label

**create DataLoader for each type of dataset**

**I need to consider the "ince" and the "rgb" data**

| Dataloader Type    | Dataloader Name |
|-------------------|-----------------|
| Train Dataloader  | 1. train_ince_dl |
|                   | 2. train_rgb_dl  |
| Val Dataloader    | 3. val_ince_dl   |
|                   | 4. val_rgb_dl    |
| Test Dataloader   | 5. test_ince_dl  |
|                   | 6. test_rgb_dl   |


In [6]:
train_ince_dl_temp       =     TrainDataset(train_dir = train_dir, label_dir = label_dir, type = "train", val_size = 0.2, sep = "ince")
train_rgb_dl_temp        =     TrainDataset(train_dir = train_dir, label_dir = label_dir, type = "train", val_size = 0.2, sep = "rgb")
val_ince_dl_temp         =     TrainDataset(train_dir = train_dir, label_dir = label_dir, type = "val", val_size = 0.2, sep = "ince")
val_rgb_dl_temp          =     TrainDataset(train_dir = train_dir, label_dir = label_dir, type = "val", val_size = 0.2, sep = "rgb")
test_ince_rgb_dl_temp    =     TestDataset(test_dir = test_dir)

In [7]:
train_ince_dl       =     DataLoader(train_ince_dl_temp, batch_size = 8)
train_rgb_dl        =     DataLoader(train_rgb_dl_temp, batch_size = 8)
val_ince_dl         =     DataLoader(val_ince_dl_temp, batch_size = 8)
val_rgb_dl          =     DataLoader(val_rgb_dl_temp, batch_size = 8)
test_ince_rgb_dl    =     DataLoader(test_ince_rgb_dl_temp, batch_size = 8)

**create Unet and training function**

*allow me to make an estimation first and inspection usnig torchsummary first*

- initiate the model

In [8]:
incemodel = UShapeNet(4).to(device=device)
# summary(incemodel, (4,350,350));
rgbmodel = UShapeNet(3).to(device=device)
# summary(rgbmodel, (3,350,350));

**define the training process!  together with our DataLoader**
- training ince model 

In [9]:
loss_fn = CombinedLoss(weight_dice=0.5, weight_ce=0.5)
optimizer = torch.optim.Adam(incemodel.parameters(), lr=0.001)
num_epochs = 5
train_accuracies = []
train_losses = []
val_accuracies = []
for epoch in trange(num_epochs, desc="out of epochs", leave=True):
    train_epoch_accuracy_list = []
    train_epoch_batch_loss = []
    val_epoch_accuracy_list = []
    for index, batch in enumerate(tqdm(iter(train_ince_dl))):
        x, y = batch
        batch_loss = train_batch(x, y, incemodel, loss_fn, optimizer)
        train_epoch_batch_loss.append(batch_loss)
        
    for index, batch in enumerate(tqdm(iter(train_ince_dl))):
        x, y = batch
        temp = accuracy(x, y, incemodel)
        train_epoch_accuracy_list.append(temp)
    
    for index, batch in enumerate(tqdm(iter(val_ince_dl))):
        x, y = batch
        temp = accuracy(x, y, incemodel)
        val_epoch_accuracy_list.append(temp)
        
    train_accuracies.append(np.mean(train_epoch_accuracy_list))
    train_losses.append(np.mean(train_epoch_batch_loss))
    val_accuracies.append(np.mean(val_epoch_accuracy_list))
    
print(f"train_accuracies is {train_accuracies}")
print(f"train_losses is {train_losses}")
print(f"val_accuracies is {val_accuracies}")

out of epochs: 100%|██████████| 5/5 [00:34<00:00,  6.98s/it]

train_accuracies is [0.4579092, 0.992591, 0.992591, 0.992591, 0.992591]
train_losses is [0.6472536444664001, 1.6129690766334535, 0.2996929168701172, 0.28164169788360593, 0.2778940796852112]
val_accuracies is [0.3937413, 0.99517804, 0.99517804, 0.99517804, 0.99517804]


- save the ince model

In [10]:
torch.save(incemodel.state_dict(), os.path.join(modelPath,'ince_model_params.pth'))

- train rgb model

In [11]:
loss_fn = CombinedLoss(weight_dice=0.5, weight_ce=0.5)
optimizer = torch.optim.Adam(rgbmodel.parameters(), lr=0.001)
num_epochs = 5
train_accuracies = []
train_losses = []
val_accuracies = []
for epoch in trange(num_epochs, desc="out of epochs", leave=True):
    train_epoch_accuracy_list = []
    train_epoch_batch_loss = []
    val_epoch_accuracy_list = []
    for index, batch in enumerate(tqdm(iter(train_rgb_dl))):
        x, y = batch
        batch_loss = train_batch(x, y, rgbmodel, loss_fn, optimizer)
        train_epoch_batch_loss.append(batch_loss)
        
    for index, batch in enumerate(tqdm(iter(train_rgb_dl))):
        x, y = batch
        temp = accuracy(x, y, rgbmodel)
        train_epoch_accuracy_list.append(temp)
    
    for index, batch in enumerate(tqdm(iter(val_rgb_dl))):
        x, y = batch
        temp = accuracy(x, y, rgbmodel)
        val_epoch_accuracy_list.append(temp)
        
    train_accuracies.append(np.mean(train_epoch_accuracy_list))
    train_losses.append(np.mean(train_epoch_batch_loss))
    val_accuracies.append(np.mean(val_epoch_accuracy_list))
    
print(f"train_accuracies is {train_accuracies}")
print(f"train_losses is {train_losses}")
print(f"val_accuracies is {val_accuracies}")

out of epochs: 100%|██████████| 5/5 [00:27<00:00,  5.57s/it]

train_accuracies is [0.59146464, 0.992591, 0.992591, 0.992591, 0.992591]
train_losses is [0.525196635723114, 1.7771829605102538, 0.28945243954658506, 0.28411559462547303, 0.2776166796684265]
val_accuracies is [0.54165256, 0.99517804, 0.99517804, 0.99517804, 0.99517804]


- save the rgb model

In [12]:
torch.save(rgbmodel.state_dict(), os.path.join(modelPath,'rgb_model_params.pth'))

**vaccuracy and loss**
- it is super good after one epic

In [13]:
train_accuracies

[0.59146464, 0.992591, 0.992591, 0.992591, 0.992591]

In [14]:
val_accuracies

[0.54165256, 0.99517804, 0.99517804, 0.99517804, 0.99517804]

- model combined the rgb and ince together!

    how to combine the (350, 350, 2) and (350, 350, 2) together??

    It is in the [test.ipynb](https://github.com/y1u2a3n4g5/Multi-Layer-Unet/blob/main/test.ipynb)

visulization of the Unet model

In [15]:
# from torchviz import make_dot
# rgbmodel.eval()
# for i in rgbmodel.parameters():
#     i.requires_grad = True
# dummy_input = torch.randn(8, 3, 350, 350).to(device=device)
# y = rgbmodel(dummy_input)
# graph = make_dot(y, params=dict(rgbmodel.named_parameters()))
# graph.render('rgbmodel', format='png')

In [16]:
from torchviz import make_dot
incemodel.eval()
for i in incemodel.parameters():
    i.requires_grad = True
dummy_input = torch.randn(8, 4, 350, 350).to(device=device)
y = incemodel(dummy_input)
graph = make_dot(y, params=dict(incemodel.named_parameters()))
graph.render('incemodel', format='png')

'incemodel.png'